# Merge the walking network and extracted crossings from osm, project sidewalk, traffic signs and curb heights

In [1]:
import geopandas as gpd
import pandas as pd
import config_azure as cf
import folium

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_30331/1699750937.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS 

In [2]:
# Import network and crossings
CRS = 'EPSG:28992'

gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=CRS)
gdf_osm_crossings = gpd.read_file(cf.output_osm_crossings).to_crs(crs=CRS)
gdf_project_sidewalk_crossings = gpd.read_file(cf.output_project_sidewalk_crossings).to_crs(crs=CRS)
gdf_traffic_sign_crossings = gpd.read_file(cf.output_traffic_sign_crossings).to_crs(crs=CRS)

gdf_curb_crossings_4cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.04.gpkg').to_crs(crs=CRS)
gdf_curb_crossings_6cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.06.gpkg').to_crs(crs=CRS)
gdf_curb_crossings_8cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.08.gpkg').to_crs(crs=CRS)

In [4]:
# Set and delete columns
gdf_osm_crossings['crossing_type'] = 'osm'
gdf_project_sidewalk_crossings['crossing_type'] = 'project_sidewalk'
gdf_traffic_sign_crossings['crossing_type'] = 'traffic_sign'

gdf_curb_crossings_4cm['curb_height_max'] = 0.04
gdf_curb_crossings_6cm['curb_height_max'] = 0.06
gdf_curb_crossings_8cm['curb_height_max'] = 0.08

gdf_curb_crossings = pd.concat([gdf_curb_crossings_4cm, gdf_curb_crossings_6cm, gdf_curb_crossings_8cm])
gdf_curb_crossings['crossing_type'] = 'curb_height'

gdf_crossings = pd.concat([gdf_osm_crossings, gdf_project_sidewalk_crossings, 
                     gdf_traffic_sign_crossings, gdf_curb_crossings])
gdf_crossings = gdf_crossings.drop(columns=['index', 'element_type', 'osmid', 'crossing', 'highway', 'kerb',
       'tactile_paving', 'button_operated', 'traffic_signals:sound',
       'traffic_signals:vibration', 'crossing:island', 'crossing_ref',
       'bicycle', 'barrier', 'crossing:markings', 'nodes', 'footway',
       'surface', 'lit', 'smoothness', 'cycleway', 'mofa', 'moped', 'oneway',
       'sidewalk', 'traffic_sign', 'traffic_calming', 'natural'])

gdf_network['crossing'] = 'No'
gdf_crossings['crossing'] = 'Yes'


# Merge walking network and crossings
gdf_network_including_crossings = pd.concat([gdf_network, gdf_crossings])

# Export network including crossings
gdf_network_including_crossings.to_file(cf.output_basic_network_including_crossings, driver='GPKG')